# Module

In [1]:
import pandas as pd
import os
import numpy as np

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Config 

In [2]:
debug = False
# if debug:
debug_num = 100
# else:
#     sample_num = -1
data_dir = '../raw_data/'
result_dir = '../result/'

submission_cols =  ['SearchId','PropertyId']
search_id = 'srch_id'
prop_id = 'prop_id'
regression_label = 'label'


In [3]:
# train_df.columns

# Data info 

In [4]:
if debug:
    train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'), nrows=debug_num)
    test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'), nrows=debug_num)
else:
    train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [5]:
print(f"train dimension: {train_df.shape}")
print(f"test dimension: {test_df.shape}")
print(f"Unique cols for train_df: {set(train_df.columns)-set(test_df.columns)}")
print(f"Unique cols for test_df: {set(test_df.columns)-set(train_df.columns)}")

train dimension: (9917530, 54)
test dimension: (6622629, 50)
Unique cols for train_df: {'click_bool', 'gross_bookings_usd', 'booking_bool', 'position'}
Unique cols for test_df: set()


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917530 entries, 0 to 9917529
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay          int64  
 19  

In [7]:
test_df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,2,2012-11-05 21:28:38,15,55,NaN,NaN,98,3105,3,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2012-11-05 21:28:38,15,55,NaN,NaN,98,6399,3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2012-11-05 21:28:38,15,55,NaN,NaN,98,7374,4,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2012-11-05 21:28:38,15,55,NaN,NaN,98,7771,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2012-11-05 21:28:38,15,55,NaN,NaN,98,12938,3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Functions

In [25]:
def save_submission(rec_df, file_name: str):
    result_df = test_df[['srch_id', 'prop_id']].rename(columns=dict(zip(['srch_id', 'prop_id'], submission_cols)))
    result_df = result_df.merge(rec_df, how='left', on=submission_cols)
    assert result_df.shape[0] == rec_df.shape[0]
    result_df = result_df.sort_values(['SearchId', "predicted"], ascending=[True, False])
    file_name = os.path.join(result_dir, file_name)
    result_df[submission_cols].to_csv(file_name, index=False)
    print(f"shape: {result_df[submission_cols].shape}")
    print(result_df.head())
    print(result_df[submission_cols].head())
    

# Random Recommenation 

## Train 

In [7]:
random_rec = test_df[['srch_id', 'prop_id']].rename(columns=dict(zip(['srch_id', 'prop_id'], submission_cols)))
random_rec['predicted'] = np.random.rand(random_rec.shape[0])

In [8]:
random_rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6622629 entries, 0 to 6622628
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   SearchId    int64  
 1   PropertyId  int64  
 2   predicted   float64
dtypes: float64(1), int64(2)
memory usage: 151.6 MB


## Evaluation 

# XGBRegressor

## Config 

In [9]:
test_size = 0.2

In [10]:
# train_df['click_bool']

In [11]:
feature_col = test_df.columns

feature_col = feature_col.drop(search_id)
feature_col = feature_col.drop(prop_id)
feature_col = feature_col.drop('date_time')
feature_col

Index(['site_id', 'visitor_location_country_id', 'visitor_hist_starrating',
       'visitor_hist_adr_usd', 'prop_country_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score', 'orig_destination_distance', 'random_bool',
       'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate',
       'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp

In [12]:
def feature_engineering(df):
    return df

def get_label(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 2
    else:
        return 0

In [13]:
train_df.columns


Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [14]:
train_df_y = train_df.apply(lambda row: get_label(row), axis=1)

In [15]:
cleaned_train_df = feature_engineering(train_df)
cleaned_test_df = feature_engineering(test_df)

In [16]:
cleaned_train_df[feature_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917530 entries, 0 to 9917529
Data columns (total 47 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   site_id                      int64  
 1   visitor_location_country_id  int64  
 2   visitor_hist_starrating      float64
 3   visitor_hist_adr_usd         float64
 4   prop_country_id              int64  
 5   prop_starrating              int64  
 6   prop_review_score            float64
 7   prop_brand_bool              int64  
 8   prop_location_score1         float64
 9   prop_location_score2         float64
 10  prop_log_historical_price    float64
 11  price_usd                    float64
 12  promotion_flag               int64  
 13  srch_destination_id          int64  
 14  srch_length_of_stay          int64  
 15  srch_booking_window          int64  
 16  srch_adults_count            int64  
 17  srch_children_count          int64  
 18  srch_room_count              int64  
 19  

In [17]:
train_X, eval_X, train_y, eval_y = train_test_split(cleaned_train_df[feature_col], train_df_y)

In [18]:
assert train_X.shape[1] == eval_X.shape[1]

In [20]:
model_params = {
    'learning_rate':0.05
    , 'n_estimators': 1000
#     'learning_rate':0.01
#     , 'n_estimators': 1000
# , 'max_depth': 8
#     , 'min_child_weight': 0.5
#     , 'gamma':1
#     , 'colsample_bytree': 0.9
#     , 'subsample': 0.9
#     , 'reg_alpha': 1

}

In [27]:
model = XGBRegressor(**model_params)

In [29]:
model

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)

In [30]:
model.fit(X=train_X, y=train_y, verbose=True, eval_metric=['mae']
                         , eval_set=[[train_X, train_y], [eval_X, eval_y]], early_stopping_rounds=20)

[0]	validation_0-mae:0.62056	validation_1-mae:0.61228
[1]	validation_0-mae:0.59756	validation_1-mae:0.59925
[2]	validation_0-mae:0.58276	validation_1-mae:0.58393
[3]	validation_0-mae:0.57361	validation_1-mae:0.57210
[4]	validation_0-mae:0.56173	validation_1-mae:0.55943
[5]	validation_0-mae:0.54533	validation_1-mae:0.54754
[6]	validation_0-mae:0.53595	validation_1-mae:0.53616
[7]	validation_0-mae:0.52736	validation_1-mae:0.52563
[8]	validation_0-mae:0.51627	validation_1-mae:0.51538
[9]	validation_0-mae:0.50399	validation_1-mae:0.50610
[10]	validation_0-mae:0.49537	validation_1-mae:0.49650
[11]	validation_0-mae:0.48889	validation_1-mae:0.48791
[12]	validation_0-mae:0.48171	validation_1-mae:0.47961
[13]	validation_0-mae:0.47313	validation_1-mae:0.47176
[14]	validation_0-mae:0.46381	validation_1-mae:0.46435
[15]	validation_0-mae:0.45601	validation_1-mae:0.45737
[16]	validation_0-mae:0.44936	validation_1-mae:0.45044
[17]	validation_0-mae:0.44409	validation_1-mae:0.44401
[18]	validation_0-ma

[149]	validation_0-mae:0.32050	validation_1-mae:0.32056
[150]	validation_0-mae:0.32049	validation_1-mae:0.32054
[151]	validation_0-mae:0.32047	validation_1-mae:0.32053
[152]	validation_0-mae:0.32045	validation_1-mae:0.32052
[153]	validation_0-mae:0.32044	validation_1-mae:0.32051
[154]	validation_0-mae:0.32042	validation_1-mae:0.32049
[155]	validation_0-mae:0.32040	validation_1-mae:0.32047
[156]	validation_0-mae:0.32039	validation_1-mae:0.32045
[157]	validation_0-mae:0.32038	validation_1-mae:0.32044
[158]	validation_0-mae:0.32037	validation_1-mae:0.32043
[159]	validation_0-mae:0.32036	validation_1-mae:0.32042
[160]	validation_0-mae:0.32034	validation_1-mae:0.32041
[161]	validation_0-mae:0.32033	validation_1-mae:0.32040
[162]	validation_0-mae:0.32031	validation_1-mae:0.32039
[163]	validation_0-mae:0.32029	validation_1-mae:0.32037
[164]	validation_0-mae:0.32028	validation_1-mae:0.32036
[165]	validation_0-mae:0.32027	validation_1-mae:0.32035
[166]	validation_0-mae:0.32026	validation_1-mae:

[296]	validation_0-mae:0.31946	validation_1-mae:0.31970
[297]	validation_0-mae:0.31945	validation_1-mae:0.31970
[298]	validation_0-mae:0.31945	validation_1-mae:0.31969
[299]	validation_0-mae:0.31945	validation_1-mae:0.31969
[300]	validation_0-mae:0.31944	validation_1-mae:0.31969
[301]	validation_0-mae:0.31943	validation_1-mae:0.31968
[302]	validation_0-mae:0.31943	validation_1-mae:0.31968
[303]	validation_0-mae:0.31943	validation_1-mae:0.31968
[304]	validation_0-mae:0.31943	validation_1-mae:0.31968
[305]	validation_0-mae:0.31942	validation_1-mae:0.31967
[306]	validation_0-mae:0.31942	validation_1-mae:0.31967
[307]	validation_0-mae:0.31941	validation_1-mae:0.31967
[308]	validation_0-mae:0.31940	validation_1-mae:0.31966
[309]	validation_0-mae:0.31940	validation_1-mae:0.31966
[310]	validation_0-mae:0.31940	validation_1-mae:0.31966
[311]	validation_0-mae:0.31940	validation_1-mae:0.31965
[312]	validation_0-mae:0.31939	validation_1-mae:0.31965
[313]	validation_0-mae:0.31939	validation_1-mae:

[443]	validation_0-mae:0.31895	validation_1-mae:0.31938
[444]	validation_0-mae:0.31894	validation_1-mae:0.31938
[445]	validation_0-mae:0.31894	validation_1-mae:0.31938
[446]	validation_0-mae:0.31894	validation_1-mae:0.31938
[447]	validation_0-mae:0.31894	validation_1-mae:0.31938
[448]	validation_0-mae:0.31893	validation_1-mae:0.31937
[449]	validation_0-mae:0.31893	validation_1-mae:0.31937
[450]	validation_0-mae:0.31893	validation_1-mae:0.31937
[451]	validation_0-mae:0.31893	validation_1-mae:0.31937
[452]	validation_0-mae:0.31892	validation_1-mae:0.31937
[453]	validation_0-mae:0.31892	validation_1-mae:0.31937
[454]	validation_0-mae:0.31892	validation_1-mae:0.31937
[455]	validation_0-mae:0.31892	validation_1-mae:0.31937
[456]	validation_0-mae:0.31892	validation_1-mae:0.31937
[457]	validation_0-mae:0.31891	validation_1-mae:0.31937
[458]	validation_0-mae:0.31891	validation_1-mae:0.31937
[459]	validation_0-mae:0.31891	validation_1-mae:0.31936
[460]	validation_0-mae:0.31890	validation_1-mae:

[590]	validation_0-mae:0.31859	validation_1-mae:0.31921
[591]	validation_0-mae:0.31859	validation_1-mae:0.31921
[592]	validation_0-mae:0.31858	validation_1-mae:0.31921
[593]	validation_0-mae:0.31858	validation_1-mae:0.31921
[594]	validation_0-mae:0.31858	validation_1-mae:0.31921
[595]	validation_0-mae:0.31858	validation_1-mae:0.31921
[596]	validation_0-mae:0.31857	validation_1-mae:0.31920
[597]	validation_0-mae:0.31857	validation_1-mae:0.31920
[598]	validation_0-mae:0.31857	validation_1-mae:0.31920
[599]	validation_0-mae:0.31857	validation_1-mae:0.31920
[600]	validation_0-mae:0.31857	validation_1-mae:0.31920
[601]	validation_0-mae:0.31857	validation_1-mae:0.31920
[602]	validation_0-mae:0.31857	validation_1-mae:0.31920
[603]	validation_0-mae:0.31856	validation_1-mae:0.31920
[604]	validation_0-mae:0.31856	validation_1-mae:0.31920
[605]	validation_0-mae:0.31856	validation_1-mae:0.31920
[606]	validation_0-mae:0.31856	validation_1-mae:0.31920
[607]	validation_0-mae:0.31855	validation_1-mae:

[737]	validation_0-mae:0.31831	validation_1-mae:0.31910
[738]	validation_0-mae:0.31830	validation_1-mae:0.31910
[739]	validation_0-mae:0.31830	validation_1-mae:0.31910
[740]	validation_0-mae:0.31830	validation_1-mae:0.31910
[741]	validation_0-mae:0.31830	validation_1-mae:0.31910
[742]	validation_0-mae:0.31830	validation_1-mae:0.31910
[743]	validation_0-mae:0.31830	validation_1-mae:0.31910
[744]	validation_0-mae:0.31829	validation_1-mae:0.31909
[745]	validation_0-mae:0.31829	validation_1-mae:0.31909
[746]	validation_0-mae:0.31829	validation_1-mae:0.31909
[747]	validation_0-mae:0.31829	validation_1-mae:0.31909
[748]	validation_0-mae:0.31829	validation_1-mae:0.31909
[749]	validation_0-mae:0.31828	validation_1-mae:0.31909
[750]	validation_0-mae:0.31828	validation_1-mae:0.31909
[751]	validation_0-mae:0.31828	validation_1-mae:0.31909
[752]	validation_0-mae:0.31827	validation_1-mae:0.31909
[753]	validation_0-mae:0.31827	validation_1-mae:0.31909
[754]	validation_0-mae:0.31827	validation_1-mae:

[884]	validation_0-mae:0.31802	validation_1-mae:0.31899
[885]	validation_0-mae:0.31802	validation_1-mae:0.31899
[886]	validation_0-mae:0.31802	validation_1-mae:0.31899
[887]	validation_0-mae:0.31802	validation_1-mae:0.31899
[888]	validation_0-mae:0.31801	validation_1-mae:0.31899
[889]	validation_0-mae:0.31801	validation_1-mae:0.31899
[890]	validation_0-mae:0.31801	validation_1-mae:0.31899
[891]	validation_0-mae:0.31801	validation_1-mae:0.31899
[892]	validation_0-mae:0.31801	validation_1-mae:0.31899
[893]	validation_0-mae:0.31801	validation_1-mae:0.31898
[894]	validation_0-mae:0.31800	validation_1-mae:0.31898
[895]	validation_0-mae:0.31800	validation_1-mae:0.31898
[896]	validation_0-mae:0.31800	validation_1-mae:0.31898
[897]	validation_0-mae:0.31799	validation_1-mae:0.31898
[898]	validation_0-mae:0.31799	validation_1-mae:0.31898
[899]	validation_0-mae:0.31799	validation_1-mae:0.31898
[900]	validation_0-mae:0.31799	validation_1-mae:0.31898
[901]	validation_0-mae:0.31799	validation_1-mae:

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [31]:
xgb_rec = test_df[['srch_id', 'prop_id']].rename(columns=dict(zip(['srch_id', 'prop_id'], submission_cols)))
xgb_rec['predicted'] = model.predict(cleaned_test_df[feature_col])

In [32]:
xgb_rec.head()

,SearchId,PropertyId,predicted
0,2,3105,0.013964
1,2,6399,0.280463
2,2,7374,0.463144
3,2,7771,0.119053
4,2,12938,0.108593


## Save 

In [34]:
save_submission(rec_df=xgb_rec, file_name='0429_xgb_v2.csv')

shape: (6622629, 2)
   SearchId  PropertyId  predicted
5         2       25579   0.480204
2         2        7374   0.463144
6         2       26540   0.400819
1         2        6399   0.280463
8         2       30434   0.245726
   SearchId  PropertyId
5         2       25579
2         2        7374
6         2       26540
1         2        6399
8         2       30434
